# ERL Analysis 2019-02-22

Code for fixing the South Sudan and missing GHS data. South Sudan, w/ FIDs duplicates and <5000 dropped
FIDS may be duplicated for GHS 2000 & 2015 because of the addition of South Sudan Data

-- Cascade Tuholske 2019-02-22

## Dependencies 

In [1]:
from rasterstats import zonal_stats
import rasterio
import geopandas as gpd
import operator
import numpy as np
import pandas as pd
import matplotlib 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from functools import reduce
import squarify

## Functions

In [2]:
def dup_drop(gpd_in, col, keep_dup):
    """ 
    function drops duplicates based on a column from a pd data frame
    requires pd df out string, pd df, col name, and which dup to keep
    returns new gpd_df
    """
    
    gpd_out = gpd.GeoDataFrame()
    
    print(gpd_in.shape)
    
    gpd_out = gpd_in.drop_duplicates(col, keep = keep_dup)
    
    print(gpd_out.shape)

    return gpd_out

## Data

In [58]:
# File Paths

data_raw = '../../data/raw/'
data_temp = '../../temp_data/'
data_interim = '../../interim/'
data_analysis = '/Users/cascade/Github/NTL/temp_data/ERL_data/Data20190222/'
downloads = '/Users/cascade/Downloads/'
desktop = '/Users/cascade/Desktop/'

In [17]:
# Load data: ...20190222.shp files have FIDs removed, S Sudan added, rainfall zones, and regions

GHS2000 = gpd.read_file(data_analysis+'GHS_POP_GPW42000_20190222.shp', driver = 'ESRI Shapefile')
GHS2000_New = gpd.read_file(data_analysis+'GHS_POP_GPW42000_urbanmerge_PopTot.shp')
GHS2000_Sudan = gpd.read_file(data_analysis+'GHS2000_1500c300_S_Sudan_polyoverlapPopTot.shp')

GHS2000_old = gpd.read_file(data_analysis+'GHS_POP_GPW42000_final20190122.shp')


In [18]:
datasets = [GHS2000, GHS2000_New, GHS2000_Sudan, GHS2000_old]

In [19]:
for dataset in datasets:
    print(len(dataset))

4220
5854
59
5854


In [66]:
Sierra_old = GHS2000_old[GHS2000_old.country == 'Egypt']
Sierra_old = Sierra_old[Sierra_old.PopTot > 5000]
Sierra_old = dup_drop(Sierra_old, 'FID', 'first')

Sierra_old.sort_values('PopTot', ascending = False)

(129, 10)
(36, 10)


,osm_id,FID,country,city,osm_type,lat,lon,PopTot,aez_class,geometry
557,270226451,4987,Egypt,???,town,30.280632,31.206051,3.098206e+07,Subtropic - warm / arid,"POLYGON ((31.50600655141665 31.35483681068133,..."
564,299573543,5306,Egypt,????,town,26.338870,31.889391,1.086632e+07,Subtropic - warm / arid,"POLYGON ((30.72878276512615 28.35776182761385,..."
589,332504177,4483,Egypt,??? ???,town,31.320302,30.064198,4.139501e+06,Subtropic - warm / arid,"POLYGON ((30.0722018254044 31.33766073549531, ..."
590,332566835,5050,Egypt,??? ????,town,29.363572,30.904181,1.652619e+06,Subtropic - warm / arid,"POLYGON ((30.82151266757599 29.50647557086778,..."
567,331953850,5084,Egypt,???,town,28.921418,30.985374,1.502062e+06,Subtropic - warm / arid,"POLYGON ((31.15595258701647 29.37860098537263,..."
618,768598369,4256,Egypt,??? ????,town,31.513239,31.833731,9.217750e+05,Subtropic - warm / arid,"POLYGON ((31.82837393679642 31.52666291819284,..."
595,376304399,4235,Egypt,???????,town,31.246623,32.320071,5.357513e+05,Subtropic - warm / arid,"POLYGON ((32.24874295584363 31.27755379410379,..."
596,395459213,4376,Egypt,???????,town,31.159012,31.936496,5.339093e+05,Subtropic - warm / arid,"POLYGON ((31.84382766475809 31.23462912591101,..."
569,331953881,5119,Egypt,?????,town,28.646183,30.840812,5.249179e+05,Subtropic - warm / arid,"POLYGON ((30.76422344052918 28.79115510899423,..."
5141,27564966,4926,Egypt,??????,city,29.974498,32.537086,3.640080e+05,Subtropic - warm / arid,"POLYGON ((32.56288050129888 30.05276542965328,..."


In [65]:
Sierra = GHS2000[GHS2000.country == 'Egypt']
Sierra.sort_values('PopTot', ascending = False)

,osm_id,FID,country,city,osm_type,lat,lon,PopTot,aez_class,dataset,region,rain_zone,geometry
401,270226451,4987,Egypt,???,town,30.280632,31.206051,3.319888e+07,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((31.50600655141665 31.35483681068133,..."
402,299573543,5306,Egypt,????,town,26.338870,31.889391,1.180503e+07,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((30.72878276512614 28.35776182761386,..."
410,332504177,4483,Egypt,??? ???,town,31.320302,30.064198,4.323961e+06,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((30.0722018254044 31.33766073549531, ..."
411,332566835,5050,Egypt,??? ????,town,29.363572,30.904181,1.748470e+06,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((30.821512667576 29.50647557086778, 3..."
404,331953850,5084,Egypt,???,town,28.921418,30.985374,1.658768e+06,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((31.15595258701646 29.37860098537263,..."
417,768598369,4256,Egypt,??? ????,town,31.513239,31.833731,9.793600e+05,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((31.82837393679642 31.52666291819284,..."
414,395459213,4376,Egypt,???????,town,31.159012,31.936496,5.720645e+05,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((31.84382766475809 31.23462912591101,..."
405,331953881,5119,Egypt,?????,town,28.646183,30.840812,5.436049e+05,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((30.76422344052918 28.79115510899423,..."
413,376304399,4235,Egypt,???????,town,31.246623,32.320071,4.839796e+05,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((32.24874295584363 31.27755379410379,..."
406,331953907,5144,Egypt,??? ????,town,28.496240,30.802898,4.143186e+05,Subtropic - warm / arid,GHS-Pop 2000,Northern_Africa,Arid,"POLYGON ((30.8552578741531 28.63811547938311, ..."


In [54]:
poly_file = 'ERL_data/GHS_POP_GPW42000_urbanmerge'
poly_gpd = gpd.read_file(data_temp+poly_file+'.shp')

In [56]:
poly_gpd.head()

,osm_id,FID,country,city,osm_type,lat,lon,geometry
0,89369215,5382,Algeria,Tamanrasset,town,22.785454,5.532446,"POLYGON ((5.512930562910484 22.80475188764244,..."
1,252600742,624,Algeria,Boumerdès,town,36.758882,3.470596,"POLYGON ((2.960095682991406 36.82071885667166,..."
2,253167052,195,Algeria,Thenia,town,36.724986,3.556935,"POLYGON ((3.610972972741118 36.75033076389462,..."
3,253167208,150,Algeria,Zemmouri,town,36.786406,3.601221,"POLYGON ((3.555512812300016 36.81191901602259,..."
4,253291208,436,Algeria,Lakhdaria,town,36.563944,3.596907,"POLYGON ((3.548917172615572 36.58325611006445,..."


In [57]:
poly_gpd.crs = {'init': 'epsg:4326'}
poly_gpd = poly_gpd.to_crs({'init': 'esri:54009'})
poly_gpd.head()

,osm_id,FID,country,city,osm_type,lat,lon,geometry
0,89369215,5382,Algeria,Tamanrasset,town,22.785454,5.532446,"POLYGON ((525405.4525564685 2791029.461868489,..."
1,252600742,624,Algeria,Boumerdès,town,36.758882,3.470596,"POLYGON ((258405.4525564685 4431029.461868489,..."
2,253167052,195,Algeria,Thenia,town,36.724986,3.556935,"POLYGON ((315405.4525564685 4423029.461868487,..."
3,253167208,150,Algeria,Zemmouri,town,36.786406,3.601221,"POLYGON ((310405.4525564685 4430029.461868489,..."
4,253291208,436,Algeria,Lakhdaria,town,36.563944,3.596907,"POLYGON ((310405.4525564685 4404029.461868488,..."


In [59]:
poly_gpd.to_file(desktop+'GHS_POP_GPW42000_urbanmerge_54009.shp', driver='ESRI Shapefile')

### Stats for Tables

In [16]:
# country = 'Mali'
# dataset1 = GHS2000
# dataset2 = GHS2015

# test = dataset1[dataset1['country'] == country]
# test = test[test.PopTot <5*10**6]

# print(test.PopTot.count())
# print(test.PopTot.median())
# print(test.PopTot.mean())
# print(test.PopTot.sum())

# test = dataset2[dataset2['country'] == country]
# test = test[test.PopTot <5*10**6]

# print(test.PopTot.count())
# print(test.PopTot.median())
# print(test.PopTot.mean())
# print(test.PopTot.sum())

157
11893.61264038086
26090.79506746523
4096254.825592041
186
13902.646545410156
41422.97556633077
7704673.455337524


### Chunk Data

### Chunk Data

### Plots

## Old Code

### Functions

In [ ]:
# # Load Data

# #GHS - Note, useing the merge files PopTot because I had to re-do the zonal stats 
# GHS2000 = gpd.read_file(data_analysis+'GHS_POP_GPW42000_urbanmerge_PopTot.shp')
# GHS2000_Sudan = gpd.read_file(data_analysis+'GHS2000_1500c300_S_Sudan_polyoverlapPopTot.shp')
# GHS2015 = gpd.read_file(data_analysis+'GHS_POP_GPW42015_urbanmerge_PopTot.shp')
# GHS2015_Sudan = gpd.read_file(data_analysis+'GHS2015_1500c300_S_Sudan_polyoverlapPopTot.shp')

# #WP
# WP2000 = gpd.read_file(data_analysis+'AFR_PPP_2000_adj_v2_final20190122.shp')
# WP2000_Sudan = gpd.read_file(data_analysis+'AFR_PPP_2015_adj_v2_S_Sudan_1500c300_polyoverlapPopTot.shp')
# WP2015 = gpd.read_file(data_analysis+'AFR_PPP_2015_adj_v2_final20190122.shp')
# WP2015_Sudan = gpd.read_file(data_analysis+'AFR_PPP_2015_adj_v2_S_Sudan_1500c300_polyoverlapPopTot.shp')


# #WPE
# WPE2016 = gpd.read_file(data_analysis+'WPE_1KM_2016_final20190122.shp')
# WPE2016_Sudan = gpd.read_file(data_analysis+'WPE_1KM_2016_Pop_Clip_S_Sudan_1500c300_polyoverlapPopTot.shp')

# #LS 
# LS2015 = gpd.read_file(data_analysis+'LS15_final20190122.shp')
# LS2015_Sudan = gpd.read_file(data_analysis+'LS15_w001001_Clip_S_Sudan_1500c300_polyoverlapPopTot.shp')

# print(len(GHS2000))
# print(len(GHS2000_Sudan))
# print(len(GHS2015))
# print(len(GHS2015_Sudan))
# print(len(WP2000))
# print(len(WP2000_Sudan))
# print(len(WP2015))
# print(len(WP2015_Sudan))
# print(len(WPE2016))
# print(len(WPE2016_Sudan))
# print(len(LS2015))
# print(len(LS2015_Sudan))


In [ ]:
# # Drop FID duplicates ... recall that FIDs for GHS will not be unique for South Sudan Dataset and GHS All 

# GHS2000 = dup_drop(GHS2000, 'FID', 'first')
# GHS2000_Sudan = dup_drop(GHS2000_Sudan, 'FID', 'first')

# GHS2015 = dup_drop(GHS2015, 'FID', 'first')
# GHS2015_Sudan = dup_drop(GHS2015_Sudan, 'FID', 'first')

# WP2000 = dup_drop(WP2000, 'FID', 'first')
# WP2000_Sudan = dup_drop(WP2000_Sudan, 'FID', 'first')

# WP2015 = dup_drop(WP2015, 'FID', 'first')
# WP2015_Sudan = dup_drop(WP2015_Sudan, 'FID', 'first')

# WPE2016 = dup_drop(WPE2016, 'FID', 'first')
# WPE2016_Sudan = dup_drop(WPE2016_Sudan, 'FID', 'first')

# LS2015 = dup_drop(LS2015, 'FID', 'first')
# LS2015_Sudan = dup_drop(LS2015_Sudan, 'FID', 'first')

# print(len(GHS2000))
# print(len(GHS2000_Sudan))
# print(len(GHS2015))
# print(len(GHS2015_Sudan))
# print(len(WP2000))
# print(len(WP2000_Sudan))
# print(len(WP2015))
# print(len(WP2015_Sudan))
# print(len(WPE2016))
# print(len(WPE2016_Sudan))
# print(len(LS2015))
# print(len(LS2015_Sudan))

In [ ]:
# Merge in South Sudan ... do each 
# print(len(GHS2015))
# print(len(GHS2015_Sudan))
# frames = [GHS2015, GHS2015_Sudan]

# GHS2015_merge = pd.concat(frames)
# print(len(GHS2015_merge))

In [ ]:
# # Drop cities with > 5,000 people

# print(len(GHS2000_merge))
# GHS2000_merge = GHS2000_merge[GHS2000_merge.PopTot >= 5000]
# print(len(GHS2000_merge))

# print(len(GHS2015_merge))
# GHS2015_merge = GHS2015_merge[GHS2015_merge.PopTot >= 5000]
# print(len(GHS2015_merge))

# print(len(WP2000_merge))
# WP2000_merge = WP2000_merge[WP2000_merge.PopTot >= 5000]
# print(len(WP2000_merge))

# print(len(WP2015_merge))
# WP2015_merge = WP2015_merge[WP2015_merge.PopTot >= 5000]
# print(len(WP2015_merge))

# print(len(LS2015_merge))
# LS2015_merge = LS2015_merge[LS2015_merge.PopTot >= 5000]
# print(len(LS2015_merge))

# print(len(WPE2016_merge))
# WPE2016_merge = WPE2016_merge[WPE2016_merge.PopTot >= 5000]
# print(len(WPE2016_merge))

In [ ]:
# Add dataset column 

# GHS2000_merge['dataset'] = 'GHS-Pop 2000'
# GHS2015_merge['dataset'] = 'GHS-Pop 2015'
# WP2000_merge['dataset'] = 'WorldPop 2000'
# WP2015_merge['dataset'] = 'WorldPop 2015'
# LS2015_merge['dataset'] = 'LandScan 2015'
# WPE2016_merge['dataset'] = 'WPE2016'

In [ ]:
# datasets = [GHS2000_merge, GHS2015_merge, WP2000_merge, WP2015_merge, LS2015_merge, WPE2016_merge]

In [ ]:
# Add Regions

### List of African Countries from the UN in OSM wiki

# Northern_Africa = (['Algeria', 'Egypt', 'Libya', 'Morocco', 'Tunisia', 'Western Sahara'], 'Northern_Africa')

# Eastern_Africa = ([
#     'Burundi',
#     'Comoros',
#     'Djibouti',
#     'Eritrea',
#     'Ethiopia',
#     'Kenya',
#     'Madagascar',
#     'Malawi',
#     'Mauritius',
#     #Mayotte,
#     'Mozambique',
#     'Réunion',
#     'Rwanda',
#     'Somalia',
#     'Sudan',
#     'South Sudan',
#     'Uganda',
#     'Tanzania',
#     'Zambia',
#     'Zimbabwe'], 'Eastern_Africa')
    
# Middle_Africa = ([
#     'Angola',
#     'Cameroon',
#     'Central African Republic',
#     'Chad',
#     'Congo-Brazzaville',
#     'Congo-Kinshasa',
#     'Equatorial Guinea',
#     'Gabon',
#     'Sao Tome and Principe'], 'Middle_Africa')
    
# Southern_Africa = ([
#     'Botswana',
#     'Lesotho',
#     'Namibia',
#     'South Africa',
#     'Swaziland'], 'Southern_Africa')
    
# Western_Africa = ([
#     'Benin',
#     'Burkina Faso',
#     'Cape Verde',
#     'Côte d\'Ivoire',
#     'Gambia',
#     'Ghana',
#     'Guinea',
#     'Guinea-Bissau',
#     'Liberia',
#     'Mali',
#     'Mauritania',
#     'Niger',
#     'Nigeria',
#     'Senegal',
#     'Sierra Leone',
#     'Togo'], 'Western_Africa')

In [ ]:
# regions = [Northern_Africa, Western_Africa, Eastern_Africa, Southern_Africa, Middle_Africa]

In [ ]:
# def region_col(gpd_df, regions_tuple, col_name_in, new_col):
#     """
#     Function searchs a col of a data frame and matches it with a list of 
#     tuples of which [0] countains a list of values to be cross referenced
#     & then makes a new col with the tuple [1] ... For example, you have a col with countries
#     and you want to make a new col listing the region that country is apart of
#     """
#     arr = []
#     for i, row in gpd_df.iterrows():
#         for region in regions_tuple:
#             for country in region[0]:
#                 if row[col_name_in] == country:
#                     name = region[1]
#                     arr.append(name)
#                     break
#     gpd_df[new_col] = arr
#     return gpd_df

In [ ]:
# for dataset in datasets:
#     dataset = region_col(dataset, regions, 'country', 'region')

In [ ]:
# Add Rainfall Zone

# Group by rainfall zone

# arid = (['Temperate / arid', 
#          'Subtropic - warm / arid', 
#          'Subtropic - cool / arid', 
#          'Tropic - warm / arid',
#          'Tropic - cool / arid'], 'Arid')

# semi_arid = (['Temperate / Semi-arid', 
#               'Subtropic - warm / semiarid', 
#               'Subtropic - cool / semiarid',
#               'Tropic - warm / semiarid', 
#               'Tropic - cool / semiarid'], 'Semi-arid')    

# sub_humid = (['Temperate / sub-humid', 
#               'Subtropic - warm / subhumid', 
#               'Subtropic - cool / subhumid',
#               'Tropic - warm / subhumid', 
#               'Tropic - cool / subhumid'], 'Sub-humid')

# humid = (['Temperate / humid', 
#           'Subtropic - warm / humid', 
#           'Subtropic - cool / humid', 
#           'Tropic - warm / humid',
#           'Tropic - cool / humid'], 'Humid')

# boreal = (['Boreal'], 'Boreal')

# na = (['NoClass', '0'], 'NA')

# rain_list = [arid, semi_arid, sub_humid, humid, boreal, na]

In [ ]:
# for dataset in datasets:
#     dataset = region_col(dataset, rain_list, 'aez_class', 'rain_zone')